In [1]:
import torch

from environment_manager import *
import networkx as nx
import warnings
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import matplotlib.cm as mcolormaps


name = "240318-online-parsing"
env = load_create_environment(name)
GHComponentTable.initialise()

Setting environment variables
Copying vanilla components
File copied successfully from C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\Grasshopper Components\240318-VanillaComponents\vanilla_components.csv to ExtractionEnvironments\240318-online-parsing\00-VanillaComponents\vanilla_components.csv.
Copying components
Copying gh files


In [2]:

illegals_dict = {
    "Bifocals": "aced9701-8be9-4860-bc37-7e22622afff4",
    "Group": "c552a431-af5b-46a9-a8a4-0fcbc27ef596",
    "Sketch": "2844fec5-142d-4381-bd5d-4cbcef6d6fed",
    "Cluster": "f31d8d7a-7536-4ac8-9c96-fde6ecda4d0a",
    "Scribble": "7f5c6c55-f846-4a08-9c9a-cfdc285cc6fe",
    "Galapogos": "e6dd2904-14bc-455b-8376-948bf2e3a7bc",
    "Jump":"86720a45-ee9e-49d6-b65e-494c06f66e59", 
    "Trigger" : "5a2b0735-ba98-4b7d-a7d3-c1dfc04475ad"
}




In [3]:
def create_and_save_graph(env:EnvironmentManager.Environment, doc, filename:Path = None, create_img:bool = False, show:bool = False):
    components = []
    # extract all the components from the document
    for obj in doc.Objects:
        try:
            components.append(GHComponent(obj))
        except TypeError:
            print(f"Error in {obj}")
            continue
    canvas = Canvas(doc, env)
    # Create a directed graph
    graph = nx.DiGraph()

    pos = {}  # Dictionary to store the positions of nodes
    cat = {}  # Dictionary to store the categories of nodes
    
    # get all the connections from the graph
    for comp in components:
        instance_id = comp.id
        component_id = comp.uid
        ins, outs = comp.get_connections(canvas)                
        pos[instance_id] = (comp.X, comp.Y)  # Store the position of the current node
        cat[instance_id] = comp.category  # Store the category of the current node

        graph.add_node(instance_id, x=comp.X, y=comp.Y, compid=component_id, category=comp.category)

        if outs:
            for connection in outs:
                source_id = connection['source']['instanceid']
                recipient_id = connection['recipient']['instanceid']
                x = comp.X
                y = comp.Y
                recipient_comp = canvas.find_object_by_guid(recipient_id)
                graph.add_node(source_id, x=x, y=y, category=comp.category)
                
                if recipient_id not in graph.nodes:
                    
                    graph.add_node(recipient_id, x=recipient_comp.X, y=recipient_comp.Y, compid=recipient_comp.uid, category=recipient_comp.category)
                    pos[recipient_id] = (x, y)  # Store the position of the connected node
                    cat[recipient_id] = recipient_comp.category  # Store the category of the connected node
                
                dist = round(torch.dist(torch.tensor([comp.X, comp.Y]), torch.tensor([recipient_comp.X, recipient_comp.Y])).item(), 1)
                graph.add_edge(source_id, 
                               recipient_id,
                               distance = dist,
                               s_paramidx=connection['source']['paramidx'], 
                               r_paramidx=connection['recipient']['paramidx'],
                               s_access=connection['source']['access'],
                               r_access=connection['recipient']['access'],
                               s_datamapping=connection['source']['dm'],
                               r_datamapping=connection['recipient']['dm'],
                               s_is_optional=int(connection['source']['optional']),
                               r_is_optional=int(connection['recipient']['optional']))
                               
    if create_img:
        # Create a figure
        fig = plt.figure(figsize=(10, 8))  # Adjust the width and height as needed
        
        # Get the unique categories
        categories = set(cat.values())
        
        # Create a color map based on the number of categories
        num_colors = len(categories)
        color_map = mcolormaps.get_cmap('viridis', num_colors)
        
        # Create a dictionary mapping categories to colors
        category_colors = {}
        for i, category in enumerate(categories):
            category_colors[category] = mcolors.to_hex(color_map(i))
        
        # Create a list of colors based on the node categories
        node_colors = [category_colors[cat[node]] for node in graph.nodes()]
        
        # Draw the graph using the stored positions and colors
        nx.draw(graph, pos=pos, with_labels=False, font_weight='bold', node_size=10, node_color=node_colors, edge_color='gray', arrowsize=5)
        
        # Create a legend
        legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=category, 
                                      markerfacecolor=color, markersize=8)
                           for category, color in category_colors.items()]
        plt.legend(handles=legend_elements, title='Categories', loc='upper right')
        plt.axis('equal')
        
        if show:
            plt.show()
        
        if filename:
            location = env.dirs['graphml']
            if isinstance(filename, Path):
                # Extract the filename without the file extension
                filename_str = str(location / filename.stem)
            else:
                filename_str = os.path.join(location, filename)
            
            try:
                plt.savefig(f"{filename_str}.png", dpi=300, bbox_inches='tight')
                print(f"Graph image saved as {filename_str}.png")
            except Exception as e:
                print(f"Error saving graph image as {filename_str}.png: {e}")
        
        plt.close(fig)
    
    if filename:
        location = env.dirs['graphml']
        if isinstance(filename, Path):
            # Extract the filename without the file extension
            filename_str = str(location / filename.stem)
        else:
            filename_str = os.path.join(location, filename)
        
        try:
            # Save the graph as a GraphML file
            nx.write_graphml(graph, f"{filename_str}.graphml")
            print(f"Graph saved as {filename_str}.graphml")
        except Exception as e:
            print(f"Error saving graph as {filename_str}.graphml: {e}")
    


In [4]:
# doc = GHProcessor.get_ghdoc(r"C:\Users\jossi\Dropbox\Office_Work\Jos\GH_Graph_Learning\TTD\Testing DataStructures\Hexagonal Bagworm.gh")
# create_and_save_graph(env=env, doc=doc, filename="Hexagonal Bagworm", create_img=True ,show=True)

In [10]:
from matplotlib import MatplotlibDeprecationWarning
error_bin = env.dirs['logs']

warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning, message="^The get_cmap function was deprecated")
folder = Path(env.dirs['processed'])
files = list(folder.glob("*.gh"))
for i, file in enumerate(files):
    print(f"Processing {file.name} ({i+1}/{len(files)})")
    # ghpp = GHDocumentPreprocessor(file)
    try:
        # doc = ghpp.process_folder_or_file(illegals_dict)
        doc = GHProcessor.get_ghdoc(str(file))
        create_and_save_graph(env, doc, filename=file)
    except Exception as e:
        print(f"Error processing {file.name}")
        print(e)
        # ghpp.move_file_to_error_bin(file, e)



Processing 003a679728d66a2ba0aa1d7a2dedc3c9fb77607e.gh (1/5639)
Graph saved as ExtractionEnvironments\240318-online-parsing\05-GraphML\003a679728d66a2ba0aa1d7a2dedc3c9fb77607e.graphml
Processing 00a3ef7d6503d31e14933d862108f402480b5361.gh (2/5639)
Graph saved as ExtractionEnvironments\240318-online-parsing\05-GraphML\00a3ef7d6503d31e14933d862108f402480b5361.graphml
Processing 026c45435f26e2a29a74b38293d0daefaf6ef977.gh (3/5639)
Graph saved as ExtractionEnvironments\240318-online-parsing\05-GraphML\026c45435f26e2a29a74b38293d0daefaf6ef977.graphml
Processing 02b2b7e6994c68ae1a63aec5c1e4b5cd0815540e.gh (4/5639)
Graph saved as ExtractionEnvironments\240318-online-parsing\05-GraphML\02b2b7e6994c68ae1a63aec5c1e4b5cd0815540e.graphml
Processing 038492c2431504172dea98afa3a469e1e60b0cf3.gh (5/5639)
Graph saved as ExtractionEnvironments\240318-online-parsing\05-GraphML\038492c2431504172dea98afa3a469e1e60b0cf3.graphml
Processing 03bdda1e1b8cbe94260a1497ba6b9098d027763e.gh (6/5639)
Graph saved as E